# **FastAPI (Servicio de Inferencia)**

Es el endpoint encargado de recibir ventanas, armar el dataframe con el orden de schema-json, llama al pipeline de SVM + Calibrador y devuelve predict_label + confianza.

LLama a su vez a policy engine en caso de que la confianza baje del 70%

**LIBRERÍAS**

In [ ]:
import os, json, joblib
import pandas as pd
from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel, Field
from starlette.middleware.cors import CORSMiddleware

#MOTOR DE REGLAS
import policy

#BASE DE DATOS
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime, timezone
from typing import Optional, Dict, Any

#SL
from river import tree, metrics
from io import BytesIO

**CONEXIÓN CON CON LA BASE DE DATOS**

In [ ]:
DATABASE_URL = os.environ["DATABASE_URL"]
def get_conn():
    return psycopg2.connect(DATABASE_URL, cursor_factory=RealDictCursor)

**Función de tiempos**

In [ ]:
def _to_utc(dt):
    # si viene sin tz, asúmelo en UTC
    return dt if dt.tzinfo else dt.replace(tzinfo=timezone.utc)

**CARGA DE ARCHIVOS NECESARIOS**

In [ ]:
MODEL_PATH = os.getenv("MODEL_PATH", "models/svm_calibrado.pkl")
POLICY_FEED_FROM_BATCH = os.getenv("POLICY_FEED_FROM_BATCH", "0") == "1"
SCHEMA_PATH = os.getenv("SCHEMA_PATH", "processing/schema.json")

try:
    MODEL = joblib.load(MODEL_PATH)  # pipeline: scaler → SVM → calibración
except Exception as e:
    raise RuntimeError(f"No pude cargar el modelo en {MODEL_PATH}: {e}")

try:
    with open(SCHEMA_PATH, "r", encoding="utf-8") as f:
        SCHEMA = json.load(f)
    FEATURES = SCHEMA["features"]
    CLASSES = getattr(MODEL, "classes_", SCHEMA.get("classes", []))
except Exception as e:
    raise RuntimeError(f"No pude cargar el schema en {SCHEMA_PATH}: {e}")

RuntimeError: No pude cargar el modelo en models/svm_calibrado.pkl: [Errno 2] No such file or directory: 'models/svm_calibrado.pkl'

**Configuraciones para conexión remota**

In [ ]:
app = FastAPI(title="HAR Online – Inference")

# CORS para que la app de Flutter pueda llamar al API
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

**Helpers**

In [ ]:
def _df_from_window_row(row: Dict[str, Any]) -> pd.DataFrame:
    if row.get("features"):
        feats: Dict[str, Any] = row["features"]
        data = {f: feats.get(f, 0.0) for f in FEATURES}
    else:
        data = {f: row.get(f, 0.0) for f in FEATURES}
    return pd.DataFrame([data], columns=FEATURES)

def _center_ts(row: Dict[str, Any]) -> float:
    st, et = row["start_time"], row["end_time"]
    c = st + (et - st) / 2
    if c.tzinfo is None:
        c = c.replace(tzinfo=timezone.utc)
    return c.timestamp()

def _svm_predict_row(row: Dict[str, Any]) -> tuple[str, float]:
    """Predicción con el SVM calibrado (idéntico a /predict_by_window)."""
    X = _df_from_window_row(row)
    proba = MODEL.predict_proba(X)[0]
    i_top = int(proba.argmax())
    pred_label = str(CLASSES[i_top])
    confianza = float(proba[i_top])
    return pred_label, confianza

def _update_window_preds(cur, win_id: int, pred_label: str, confianza: float) -> None:
    cur.execute(
        """
        UPDATE windows
           SET pred_label = %s,
               confianza  = %s
         WHERE id = %s
        """,
        (pred_label, confianza, win_id),
    )

**Models**

In [ ]:
class PredictByWindowReq(BaseModel):
    id: int = Field(..., description="PK de windows")

class LabelReq(BaseModel):
    id_usuario: int
    session_id: int
    start_ts: datetime
    end_ts: datetime
    label: str
    reason: Optional[str] = None
    pending_id: int | None = None

class MarkReq(BaseModel):
    id_usuario: int
    session_id: int
    when: Optional[datetime] = None

class PredictPendingReq(BaseModel):
    limit: int = 200
    id_usuario: Optional[int] = None
    session_id: Optional[int] = None

**Gets**


Realiza un monitoreo para corroborar que el servidor esté en producción y haya cargado los archivos.

In [ ]:
@app.get("/health")
def health():
    return {"ok": True, "model": os.path.basename(MODEL_PATH), "n_features": len(FEATURES)}

@app.get("/schema")
def schema():
    return {"features": FEATURES, "classes": list(map(str, CLASSES))}

**Endpoint de Ingest, Entrenamiento y Evaluación de Policy**

In [ ]:
@app.post("/predict_by_window")
def predict_by_window(req: PredictByWindowReq):
    global SL_UPDATES, SL_PROMOTED, SL2_UPDATES, SL2_PROMOTED
    with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
        # 1) Leer ventana
        cur.execute("SELECT * FROM windows WHERE id = %s", (req.id,))
        w = cur.fetchone()
        if not w:
            raise HTTPException(404, f"id {req.id} no existe")

        # 2) Predicción SVM calibrado (siempre: SVM -> pred_label/confianza)
        pred_label, confianza = _svm_predict_row(w)

        uid = int(w["id_usuario"])
        sid = int(w["session_id"])
        center_ts = _center_ts(w)

        # Semilla de policy: último label confirmado (uid, sid)
        cur.execute("""
            SELECT COALESCE(EXTRACT(EPOCH FROM end_ts), EXTRACT(EPOCH FROM created_at)) AS ts, label
              FROM intervalos_label
             WHERE id_usuario = %s AND session_id = %s AND label IS NOT NULL
             ORDER BY COALESCE(end_ts, created_at) DESC
             LIMIT 1
        """, (uid, sid))
        seed = cur.fetchone()
        if seed:
            policy.mark_labeled(uid, sid, now_ts=float(seed["ts"]), label=seed["label"])

        # 3) SL/SL2: entrenamiento oportunista con ventanas etiquetadas pendientes (A y/o B)
        trained_A = 0
        trained_B = 0

        want_op_A = (SL_ENABLED and SL_LEARNER is not None and os.getenv("SL_OPPORTUNISTIC", "0") == "1")
        want_op_B = (SL2_ENABLED and SL2_LEARNER is not None and os.getenv("SL2_OPPORTUNISTIC", "0") == "1")

        if want_op_A or want_op_B:
            cur.execute("""
                SELECT id, features, etiqueta, *
                  FROM windows
                WHERE id_usuario = %s
                  AND session_id  = %s
                  AND etiqueta IS NOT NULL
                  AND etiqueta <> ''
                  AND etiqueta NOT IN ('null','NULL','None')
                  AND (
                        COALESCE(sl_trained, FALSE) = FALSE
                      OR ( %s = '1' AND COALESCE(sl_trained_b, FALSE) = FALSE )
                  )
                ORDER BY id ASC
                LIMIT %s
                FOR UPDATE SKIP LOCKED
            """, (uid, sid, os.getenv("SL2_OPPORTUNISTIC", "0"), SL_TRAIN_CHUNK))
            rows_train = cur.fetchall()

            for r in rows_train:
                x = _row_to_x(r); y = r["etiqueta"]

                # --- A: prequential + learn
                if want_op_A:
                    try:
                        y_hat = SL_LEARNER.predict_one(x)
                        if y_hat is not None:
                            SL_METRIC.update(y_true=y, y_pred=y_hat)
                    except Exception:
                        pass
                    SL_LEARNER.learn_one(x, y)
                    SL_UPDATES += 1
                    trained_A += 1

                # --- B: prequential + learn
                if want_op_B:
                    try:
                        y_hat2 = SL2_LEARNER.predict_one(x)
                        if y_hat2 is not None:
                            SL2_METRIC.update(y_true=y, y_pred=y_hat2)
                    except Exception:
                        pass
                    SL2_LEARNER.learn_one(x, y)
                    SL2_UPDATES += 1
                    trained_B += 1

                # Marca como entrenado para evitar reuso infinitum.
                if want_op_A and want_op_B:
                    cur.execute("UPDATE windows SET sl_trained = TRUE, sl_trained_b = TRUE WHERE id = %s", (r["id"],))
                elif want_op_A:
                    cur.execute("UPDATE windows SET sl_trained = TRUE WHERE id = %s", (r["id"],))
                elif want_op_B:
                    cur.execute("UPDATE windows SET sl_trained_b = TRUE WHERE id = %s", (r["id"],))

            # Snapshots (disparo por cruce de múltiplos)
            if trained_A and (SL_UPDATES // SL_SNAPSHOT_EVERY > (SL_UPDATES - trained_A) // SL_SNAPSHOT_EVERY):
                _sl_snapshot(conn, promoted=False)
            if trained_B and (SL2_UPDATES // SL2_SNAPSHOT_EVERY > (SL2_UPDATES - trained_B) // SL2_SNAPSHOT_EVERY):
                _sl2_snapshot(conn, promoted=False)

        # 4) Motor de políticas (preguntar etiqueta si baja confianza, cambio, keep-alive, etc.)
        ask, ask_reason = policy.should_ask(
            uid, sid, confianza, pred_label=pred_label, now_ts=center_ts
        )

        # 5) Guardar predicción SVM
        cur.execute(
            "UPDATE windows SET pred_label=%s, confianza=%s WHERE id=%s",
            (pred_label, confianza, req.id),
        )

        ask_id = None
        if ask:
            policy.mark_asked(uid, sid, center_ts, ask_reason)
            # UPDATE-then-INSERT de la pendiente (sin tocar start/end aquí)
            cur.execute("""
                UPDATE intervalos_label
                   SET reason = %s,
                       created_at = NOW()
                 WHERE id_usuario = %s
                   AND session_id  = %s
                   AND label IS NULL
                 RETURNING id
            """, (ask_reason, uid, sid))
            row_upd = cur.fetchone()
            if row_upd:
                ask_id = row_upd["id"]
            else:
                cur.execute("""
                    INSERT INTO intervalos_label
                        (id_usuario, session_id, label, reason, created_at)
                    VALUES (%s, %s, NULL, %s, NOW())
                    RETURNING id
                """, (uid, sid, ask_reason))
                ask_id = cur.fetchone()["id"]

        # 6) SL en sombra o “principal”: siempre escribir actividad/precision si ya calentó
        sl_pred, sl_conf = None, None
        if SL_ENABLED and SL_LEARNER is not None and SL_UPDATES >= SL_MIN_SHADOW_UPDATES:
            x = _row_to_x(w)
            try:
                sl_pred = SL_LEARNER.predict_one(x)
                proba   = SL_LEARNER.predict_proba_one(x) or {}
                sl_conf = float(proba.get(sl_pred, 0.0)) if sl_pred is not None else None
            except Exception:
                sl_pred, sl_conf = None, None

            cur.execute("""
                UPDATE windows
                   SET actividad = %s,
                       precision = %s
                 WHERE id = %s
            """, (sl_pred, sl_conf, req.id))

        # === SL-B en sombra ===
        sl2_pred, sl2_conf = None, None
        if SL2_ENABLED and SL2_LEARNER is not None and SL2_UPDATES >= SL2_MIN_SHADOW_UPDATES:
            x = _row_to_x(w)
            try:
                sl2_pred = SL2_LEARNER.predict_one(x)
                proba2   = SL2_LEARNER.predict_proba_one(x) or {}
                sl2_conf = float(proba2.get(sl2_pred, 0.0)) if sl2_pred is not None else None
            except Exception:
                sl2_pred, sl2_conf = None, None

            cur.execute("""
                UPDATE windows
                  SET actividad_b = %s,
                      precision_b = %s
                WHERE id = %s
            """, (sl2_pred, sl2_conf, req.id))

        # 7) Promoción automática de SL (flag + snapshot) si cumple umbrales
        if SL_ENABLED and not SL_PROMOTED and SL_UPDATES >= SL_PROMOTE_MIN_UPDATES:
            try:
                acc_a = float(SL_METRIC.get() or 0.0)
            except Exception:
                acc_a = 0.0
            if acc_a >= SL_PROMOTE_MIN_ACC:
                SL_PROMOTED = True
                _sl_snapshot(conn, promoted=True)

        # B
        if SL2_ENABLED and not SL2_PROMOTED and SL2_UPDATES >= SL2_PROMOTE_MIN_UPDATES:
            try:
                acc_b = float(SL2_METRIC.get() or 0.0)
            except Exception:
                acc_b = 0.0
            if acc_b >= SL2_PROMOTE_MIN_ACC:
                SL2_PROMOTED = True
                _sl2_snapshot(conn, promoted=True)


    return {
        "id": req.id,
        "pred_label": pred_label,       # SVM (sombra si SL ya promovido)
        "confianza": confianza,         # SVM
        "actividad": sl_pred,           # SL (sea sombra o principal, siempre aquí)
        "precision": sl_conf,           # SL
        "actividad_b": sl2_pred,
        "precision_b": sl2_conf,
        "ask_label": ask,
        "ask_reason": ask_reason,
        "ask_id": ask_id,
        "sl_shadow": {
            "algo": SL_ALGO,
            "updates": SL_UPDATES,
            "promoted": SL_PROMOTED
        },
        "sl_shadow_b": {
            "algo": SL2_ALGO,
            "updates": SL2_UPDATES,
            "promoted": SL2_PROMOTED
        },
        "classes": list(map(str, CLASSES)),
    }

**Endpoint de prediccion pendiente (Reserva por filas con prediccion pendiente)**

In [ ]:
@app.post("/predict_pending")
def predict_pending(req: PredictPendingReq):
    """
    Procesa ventanas donde pred_label es NULL.
    Puedes filtrar por id_usuario y/o session_id para un barrido específico.
    """
    done = 0

    # Construimos WHERE dinámico
    where = ["pred_label IS NULL"]
    params: list[Any] = []
    if req.id_usuario is not None:
        where.append("id_usuario = %s")
        params.append(req.id_usuario)
    if req.session_id is not None:
        where.append("session_id = %s")
        params.append(req.session_id)
    where_sql = " AND ".join(where)

    with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
        # Tomamos un lote estable y lo bloqueamos para evitar colisiones
        sql_sel = f"""
            SELECT *
              FROM windows
             WHERE {where_sql}
             ORDER BY id ASC
             LIMIT %s
             FOR UPDATE SKIP LOCKED
        """
        cur.execute(sql_sel, (*params, req.limit))
        rows = cur.fetchall()

        for row in rows:
            win_id = int(row["id"])
            # si no hay features JSON, _df_from_window_row usará columnas sueltas
            try:
                pred, conf = _svm_predict_row(row)
                _update_window_preds(cur, win_id, pred, conf)
                done += 1
            except Exception as e:
                # si una fila rara rompe, saltamos y seguimos con las demás
                # (opcional: loggear e)
                continue

        conn.commit()

    return {"processed": done}

**Endpoint para registrar que el usuario respondió una etiqueta**

In [ ]:
@app.post("/label")
def post_label(req: LabelReq):
    global SL_UPDATES, SL_METRIC, SL_LEARNER, SL2_UPDATES
    st = req.start_ts.astimezone(timezone.utc)
    et = req.end_ts.astimezone(timezone.utc)
    if et <= st:
        raise HTTPException(400, "end_ts debe ser > start_ts")

    with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
        # 1) Cerrar/crear intervalo (si hay pendiente, actualízalo; si no, inserta)
        cur.execute("""
            SELECT id
              FROM intervalos_label
             WHERE id_usuario = %s
               AND session_id  = %s
               AND label IS NULL
             ORDER BY created_at DESC
             LIMIT 1
             FOR UPDATE
        """, (req.id_usuario, req.session_id))
        pend = cur.fetchone()

        if pend:
            cur.execute("""
                UPDATE intervalos_label
                   SET label=%s, reason=%s, start_ts=%s, end_ts=%s, created_at=NOW()
                 WHERE id=%s
                RETURNING id
            """, (req.label, req.reason, st, et, pend["id"]))
            interval_id = cur.fetchone()["id"]
        else:
            cur.execute("""
                INSERT INTO intervalos_label
                    (start_ts, end_ts, label, reason, created_at, id_usuario, session_id)
                VALUES (%s, %s, %s, %s, NOW(), %s, %s)
                RETURNING id
            """, (st, et, req.label, req.reason, req.id_usuario, req.session_id))
            interval_id = cur.fetchone()["id"]

        # 2.a) (nuevo) rollback si el tramo ya fue entrenado (A o B)
        cur.execute("""
          SELECT
            SUM(CASE WHEN COALESCE(sl_trained, FALSE) THEN 1 ELSE 0 END) AS n_a,
            SUM(CASE WHEN COALESCE(sl_trained_b, FALSE) THEN 1 ELSE 0 END) AS n_b
          FROM windows
          WHERE id_usuario=%s AND session_id=%s
            AND start_time < %s AND end_time > %s
        """, (req.id_usuario, req.session_id, et, st))
        row_cnt = cur.fetchone() or {"n_a": 0, "n_b": 0}
        n_trained_any = int(row_cnt["n_a"] or 0) + int(row_cnt["n_b"] or 0)

        if n_trained_any > 0:
            # Reload A
            try:
                cur.execute("SELECT model_bytes FROM sl_models ORDER BY id DESC LIMIT 1")
                snap_a = cur.fetchone()
                if snap_a and snap_a["model_bytes"]:
                    SL_LEARNER = joblib.load(BytesIO(snap_a["model_bytes"]))
                    SL_METRIC = metrics.Accuracy()
            except Exception:
                pass

            # Reload B (si está habilitado)
            if SL2_ENABLED:
                try:
                    cur.execute("SELECT model_bytes FROM sl_models_b ORDER BY id DESC LIMIT 1")
                    snap_b = cur.fetchone()
                    if snap_b and snap_b["model_bytes"]:
                        SL2_LEARNER = joblib.load(BytesIO(snap_b["model_bytes"]))
                        SL2_METRIC = metrics.Accuracy()
                except Exception:
                    pass

            # Limpia flags A y B en el rango
            cur.execute("""
              UPDATE windows
                SET sl_trained = FALSE,
                    sl_trained_b = FALSE
              WHERE id_usuario=%s AND session_id=%s
                AND start_time < %s AND end_time > %s
            """, (req.id_usuario, req.session_id, et, st))


    # ENTRENAMIENTO ONLINE AL RECIBIR UNA ETIQUETA
        # 2.b) ENTRENAR SL con todas las ventanas cubiertas por [st, et]
        if SL_ENABLED and SL_LEARNER is not None:
            cur.execute("""
                SELECT id, features, *
                  FROM windows
                 WHERE id_usuario = %s
                   AND session_id  = %s
                   AND start_time < %s
                   AND end_time   > %s
                   AND COALESCE(sl_trained, FALSE) = FALSE
            """, (req.id_usuario, req.session_id, et, st))
            ws = cur.fetchall()

            for wrow in ws:
                x = _row_to_x(wrow)
                y = req.label

                # A: prequential + learn
                try:
                    y_hat = SL_LEARNER.predict_one(x)
                    if y_hat is not None:
                        SL_METRIC.update(y_true=y, y_pred=y_hat)
                except Exception:
                    pass
                SL_LEARNER.learn_one(x, y)
                SL_UPDATES += 1

                # B: prequential + learn (si aplica)
                if SL2_ENABLED and SL2_LEARNER is not None:
                    try:
                        y_hat2 = SL2_LEARNER.predict_one(x)
                        if y_hat2 is not None:
                            SL2_METRIC.update(y_true=y, y_pred=y_hat2)
                    except Exception:
                        pass
                    SL2_LEARNER.learn_one(x, y)
                    SL2_UPDATES += 1

                # Marcar entrenado(s) UNA SOLA VEZ según lo que haya aprendido
                if SL2_ENABLED and SL2_LEARNER is not None:
                    cur.execute(
                        "UPDATE windows SET sl_trained = TRUE, sl_trained_b = TRUE WHERE id = %s",
                        (wrow['id'],)
                    )
                else:
                    cur.execute(
                        "UPDATE windows SET sl_trained = TRUE WHERE id = %s",
                        (wrow['id'],)
                    )

            # snapshot periódico
            if SL_UPDATES // SL_SNAPSHOT_EVERY > (SL_UPDATES - len(ws)) // SL_SNAPSHOT_EVERY:
                _sl_snapshot(conn, promoted=False)

            # snapshot periódico
            if SL2_UPDATES // SL2_SNAPSHOT_EVERY > (SL2_UPDATES - len(ws)) // SL2_SNAPSHOT_EVERY:
                _sl2_snapshot(conn, promoted=False)

        # 2) Commit en la misma transacción
        conn.commit()

    policy.mark_labeled(req.id_usuario, req.session_id, et.timestamp(), req.label)
    return {"ok": True, "interval_id": interval_id}

In [ ]:
@app.post("/policy/asked")
def policy_mark_asked(req: MarkReq):
    policy.mark_asked(req.id_usuario, req.session_id, req.when.timestamp() if req.when else None)
    return {"ok": True}

@app.post("/policy/labeled")
def policy_mark_labeled(req: MarkReq):
    policy.mark_labeled(req.id_usuario, req.session_id, req.when.timestamp() if req.when else None)
    return {"ok": True}

**STREAM LEARNING**

Cargar o crear el learner

In [ ]:
@app.on_event("startup")
def _startup():
    global SL_LEARNER, SL_METRIC, SL_UPDATES
    global SL2_LEARNER, SL2_METRIC, SL2_UPDATES

    if not SL_ENABLED:
        return
    # Crear learner (HT online)
    SL_LEARNER = _sl_make_learner()
    SL_METRIC = metrics.Accuracy()
    SL_UPDATES = 0

    # Intentar cargar snapshot más reciente
    try:
        with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute("""
                SELECT algo, n_updates, metric, model_bytes
                  FROM sl_models
                 ORDER BY id DESC
                 LIMIT 1
            """)
            snap = cur.fetchone()
            if snap:
                model_bytes = snap["model_bytes"]
                SL_LEARNER = joblib.load(BytesIO(model_bytes))
                SL_UPDATES = int(snap["n_updates"])
                # métrica es informativa; Accuracy se resetea en tiempo real
    except Exception as e:
        # si no hay snapshot, seguimos con el árbol vacío
        pass

    if SL2_ENABLED:
      SL2_LEARNER = _sl2_make_learner()
      SL2_METRIC = metrics.Accuracy()
      SL2_UPDATES = 0
      try:
          with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
              cur.execute("""SELECT algo, n_updates, metric, model_bytes
                              FROM sl_models_b ORDER BY id DESC LIMIT 1""")
              snap2 = cur.fetchone()
              if snap2 and snap2["model_bytes"]:
                  SL2_LEARNER = joblib.load(BytesIO(snap2["model_bytes"]))
                  SL2_UPDATES = int(snap2["n_updates"] or 0)
      except Exception:
          pass

**Configuración de SL**

In [ ]:
# ======== Modelo A (HT) ==========
SL_ENABLED: bool = os.getenv("SL_ENABLED", "1") == "1"
SL_ALGO: str = os.getenv("SL_ALGO", "HT")  # por ahora solo HT
SL_MIN_SHADOW_UPDATES: int = int(os.getenv("SL_MIN_SHADOW_UPDATES", "50"))
SL_SNAPSHOT_EVERY: int = int(os.getenv("SL_SNAPSHOT_EVERY", "1000"))
# Umbrales para promover SL a "principal"
SL_PROMOTE_MIN_UPDATES = int(os.getenv("SL_PROMOTE_MIN_UPDATES", "500"))
SL_PROMOTE_MIN_ACC     = float(os.getenv("SL_PROMOTE_MIN_ACC", "0.90"))


SL_LEARNER = None           # modelo online (HT)
SL_METRIC = metrics.Accuracy()
SL_UPDATES = 0              # # de learn_one() hechos
SL_PROMOTED = False         # si SL desplaza al SVM (solo para respuesta a la app)
SL_TRAIN_CHUNK = int(os.getenv("SL_TRAIN_CHUNK", "500"))

# ===== Modelo B (ARF) =====
SL2_ENABLED: bool = os.getenv("SL2_ENABLED", "0") == "1"
SL2_ALGO: str = os.getenv("SL2_ALGO", "ARF")  # prueba ARF; fallback a HT
SL2_MIN_SHADOW_UPDATES: int = int(os.getenv("SL2_MIN_SHADOW_UPDATES", "50"))
SL2_SNAPSHOT_EVERY: int = int(os.getenv("SL2_SNAPSHOT_EVERY", "1000"))
# Umbrales para promover SL-B a "principal"
SL2_PROMOTE_MIN_UPDATES = int(os.getenv("SL2_PROMOTE_MIN_UPDATES", "500"))
SL2_PROMOTE_MIN_ACC     = float(os.getenv("SL2_PROMOTE_MIN_ACC", "0.90"))

SL2_LEARNER = None
SL2_METRIC = metrics.Accuracy()
SL2_UPDATES = 0
SL2_PROMOTED = False

def _sl_make_learner():
    """Construye el learner de SL según SL_ALGO, con fallback seguro a HT."""
    algo = (SL_ALGO or "HT").upper()

    if algo == "ARF":
        try:
            # River >= 0.15+ (actual): ARFClassifier en forest
            try:
                from river.forest import ARFClassifier as _ARF
            except Exception:
                try:
                    # Algunas versiones mantienen el nombre largo en forest
                    from river.forest import AdaptiveRandomForestClassifier as _ARF
                except Exception:
                    # Muy legacy: nombre largo en ensemble
                    from river.ensemble import AdaptiveRandomForestClassifier as _ARF

            return _ARF(
                n_models=12,
                max_features="sqrt",
                lambda_value=8,
            )
        except Exception as e:
            print("WARN: ARF no disponible → usando HT. Detalle:", repr(e))

    # Fallback: Hoeffding Tree
    return tree.HoeffdingTreeClassifier(
        grace_period=30,
        delta=1e-5,
        leaf_prediction="nb",
    )


def _sl2_make_learner():
    """Construye el learner B (SL2) con ARF si está pedido, si no HT distinto al A."""
    algo = (SL2_ALGO or "HT").upper()

    if algo == "ARF":
        try:
            # River >= 0.15+ (actual): ARFClassifier en forest
            try:
                from river.forest import ARFClassifier as _ARF
            except Exception:
                try:
                    from river.forest import AdaptiveRandomForestClassifier as _ARF
                except Exception:
                    from river.ensemble import AdaptiveRandomForestClassifier as _ARF

            return _ARF(
                n_models=12,
                max_features="sqrt",
                lambda_value=8,
            )
        except Exception as e:
            print("WARN: SL2 ARF no disponible → usando HT. Detalle:", repr(e))

    # Fallback: HT con hiperparámetros distintos al A
    return tree.HoeffdingTreeClassifier(
        grace_period=30,
        delta=1e-5,
        leaf_prediction="nb",
    )

**Función snapshot**

In [ ]:
def _sl_snapshot(conn, promoted: bool = False):
    if not SL_ENABLED or SL_LEARNER is None:
        return
    buf = BytesIO(); joblib.dump(SL_LEARNER, buf, compress=3); buf.seek(0)  # <-- compress
    acc = float(SL_METRIC.get()) if SL_METRIC is not None else 0.0
    metric_json = {"accuracy": acc}
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO sl_models (algo, promoted, n_updates, metric, model_bytes)
            VALUES (%s, %s, %s, %s, %s)
        """, (SL_ALGO, promoted, SL_UPDATES, json.dumps(metric_json), psycopg2.Binary(buf.read())))
    conn.commit()

def _sl2_snapshot(conn, promoted: bool = False):
    if not SL2_ENABLED or SL2_LEARNER is None:
        return
    buf = BytesIO(); joblib.dump(SL2_LEARNER, buf, compress=3); buf.seek(0)  # <-- compress
    acc = float(SL2_METRIC.get()) if SL2_METRIC is not None else 0.0
    metric_json = {"accuracy": acc}
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO sl_models_b (algo, promoted, n_updates, metric, model_bytes)
            VALUES (%s, %s, %s, %s, %s)
        """, (SL2_ALGO, promoted, SL2_UPDATES, json.dumps(metric_json), psycopg2.Binary(buf.read())))
    conn.commit()

**Helpers**

In [ ]:
def _row_to_x(row: Dict[str, Any]) -> Dict[str, float]:
    """Convierte fila windows → dict(feature -> valor) respetando SCHEMA."""
    if row.get("features"):
        feats: Dict[str, Any] = row["features"]
        return {f: float(feats.get(f, 0.0)) for f in FEATURES}
    return {f: float(row.get(f, 0.0)) for f in FEATURES}

**Endpoint para entrenar**

In [ ]:
@app.post("/sl_train_pending")
def sl_train_pending(req: PredictPendingReq):
    global SL_UPDATES, SL2_UPDATES
    """
    Entrena en batch con ventanas etiquetadas aún no usadas por SL (A y opcionalmente B).
    Respeta SL_TRAIN_CHUNK para evitar bloqueos largos.
    """
    if not SL_ENABLED or SL_LEARNER is None:
        return {"trained": 0, "updates": SL_UPDATES, "snapshot": False}

    trained = 0
    trained_2 = 0
    snap = False

    with get_conn() as conn, conn.cursor(cursor_factory=RealDictCursor) as cur:
        # Selecciona SOLO ventanas etiquetadas que aún no han sido usadas por SL
        # (y bloquea filas para evitar condiciones de carrera)
        freeze_secs = int(os.getenv("SL_TRAIN_FREEZE_S", "10"))

        cur.execute("""
            SELECT id, features, etiqueta, id_usuario, session_id, start_time, end_time
              FROM windows
            WHERE etiqueta IS NOT NULL
              AND etiqueta <> ''
              AND etiqueta NOT IN ('null','NULL','None')
              AND (
                    COALESCE(sl_trained, FALSE) = FALSE
                 OR ( %s = '1' AND COALESCE(sl_trained_b, FALSE) = FALSE )
              )
              AND end_time < NOW() - ( %s || ' seconds')::interval
              AND EXISTS (
                    SELECT 1
                      FROM intervalos_label i
                    WHERE i.id_usuario = id_usuario
                      AND i.session_id = session_id
                      AND i.label IS NOT NULL
                      AND (
                            (i.start_ts IS NOT NULL AND i.end_ts IS NOT NULL
                             AND start_time < i.end_ts AND end_time > i.start_ts)
                           OR
                            (i.duracion IS NOT NULL AND i.created_at IS NOT NULL
                             AND end_time BETWEEN (i.created_at - (i.duracion::int * INTERVAL '1 second'))
                                             AND  i.created_at)
                          )
                )
            ORDER BY id ASC
            LIMIT %s
            FOR UPDATE SKIP LOCKED
        """, ('1' if SL2_ENABLED and SL2_LEARNER is not None else '0', str(freeze_secs), SL_TRAIN_CHUNK))

        rows = cur.fetchall()
        if not rows:
            return {"trained": 0, "updates": SL_UPDATES, "snapshot": False}

        for r in rows:
            x = _row_to_x(r)
            y = r["etiqueta"]

            # (opcional) evaluar antes de aprender para actualizar métrica (prequential)
            try:
                y_hat = SL_LEARNER.predict_one(x)
                if y_hat is not None:
                    SL_METRIC.update(y_true=y, y_pred=y_hat)
            except Exception:
                pass

            # Aprender con A
            SL_LEARNER.learn_one(x, y)
            SL_UPDATES += 1
            trained += 1

            # Aprender con B (si está habilitado)
            if SL2_ENABLED and SL2_LEARNER is not None:
                try:
                    y_hat2 = SL2_LEARNER.predict_one(x)
                    if y_hat2 is not None:
                        SL2_METRIC.update(y_true=y, y_pred=y_hat2)
                except Exception:
                    pass
                SL2_LEARNER.learn_one(x, y)
                SL2_UPDATES += 1
                trained_2 += 1

            # Marcar la fila según qué modelos aprendieron
            if SL2_ENABLED and SL2_LEARNER is not None:
                cur.execute(
                    "UPDATE windows SET sl_trained = TRUE, sl_trained_b = TRUE WHERE id = %s",
                    (r["id"],)
                )
            else:
                cur.execute(
                    "UPDATE windows SET sl_trained = TRUE WHERE id = %s",
                    (r["id"],)
                )

        # Snapshot periódico A
        if trained and (SL_UPDATES // SL_SNAPSHOT_EVERY > (SL_UPDATES - trained) // SL_SNAPSHOT_EVERY):
            _sl_snapshot(conn, promoted=False)
            snap = True

        # Snapshot periódico B
        if trained_2 and (SL2_UPDATES // SL2_SNAPSHOT_EVERY > (SL2_UPDATES - trained_2) // SL2_SNAPSHOT_EVERY):
            _sl2_snapshot(conn, promoted=False)
            snap = True

    return {"trained": trained, "updates": SL_UPDATES, "snapshot": snap}

**Endpoint de estado y promoción**

In [ ]:
@app.get("/sl_status")
def sl_status():
    return {
        "algo": SL_ALGO,
        "promoted": SL_PROMOTED,
        "metric": {"accuracy": SL_METRIC.get()},
        "n_updates": SL_UPDATES
    }

@app.get("/sl_status_all")
def sl_status_all():
    return {
        "A": {
            "algo": SL_ALGO,
            "promoted": SL_PROMOTED,
            "metric": {"accuracy": SL_METRIC.get()},
            "n_updates": SL_UPDATES
        },
        "B": {
            "algo": SL2_ALGO,
            "enabled": SL2_ENABLED,
            "metric": {"accuracy": SL2_METRIC.get() if SL2_ENABLED else None},
            "n_updates": SL2_UPDATES if SL2_ENABLED else None
        }
    }


@app.post("/sl_promote")
def sl_promote(which: str = Query("auto", pattern="^(A|B|auto)$")):
    global SL_PROMOTED, SL2_PROMOTED

    with get_conn() as conn:
        if which == "A":
            SL_PROMOTED = True
            _sl_snapshot(conn, promoted=True)
            return {"ok": True, "promoted": "A"}
        elif which == "B":
            if not SL2_ENABLED or SL2_LEARNER is None:
                raise HTTPException(400, "SL2 no está habilitado")
            SL2_PROMOTED = True
            _sl2_snapshot(conn, promoted=True)
            return {"ok": True, "promoted": "B"}
        else:
            # auto: promueve el que cumpla mejor los umbrales actuales
            acc_a = float(SL_METRIC.get() or 0.0) if SL_ENABLED and SL_LEARNER else -1.0
            acc_b = float(SL2_METRIC.get() or 0.0) if SL2_ENABLED and SL2_LEARNER else -1.0

            cand = None
            if (acc_a >= SL_PROMOTE_MIN_ACC and SL_UPDATES >= SL_PROMOTE_MIN_UPDATES):
                cand = ("A", acc_a)
            if (acc_b >= SL2_PROMOTE_MIN_ACC and SL2_UPDATES >= SL2_PROMOTE_MIN_UPDATES):
                if cand is None or acc_b > cand[1]:
                    cand = ("B", acc_b)

            if cand is None:
                raise HTTPException(400, "Ningún modelo cumple condiciones de promoción")

            if cand[0] == "A":
                SL_PROMOTED = True
                _sl_snapshot(conn, promoted=True)
            else:
                SL2_PROMOTED = True
                _sl2_snapshot(conn, promoted=True)

            return {"ok": True, "promoted": cand[0]}

@app.post("/sl_reset")
def sl_reset():
    global SL_LEARNER, SL_METRIC, SL_PROMOTED
    SL_LEARNER = _sl_make_learner()
    SL_METRIC = metrics.Accuracy()
    SL_PROMOTED = False
    with get_conn() as conn:
        _sl_snapshot(conn, promoted=False)  # <-- en vez de _sl_save
    return {"ok": True}